In [19]:
# Mark Eilers
# CS 340: Client/Server Development
# CS 340: Project 2
# December 7, 2023

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64


# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd

# Import the MongoClient from the pymongo.
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# Import the animalShelter filte.
import animal_shelter


# Importing AnimalShelter class 
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter()


# class read method must support return of cursor object 
# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
# create dash application

externalStylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, externalStylesheets=externalStylesheets)

#FIX ME Add in Grazioso Salvare's logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design.
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Mark Eilers Project 2 Dashboard'))),
    html.Hr(),
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )

#FIXME Add in code for the interactice filtering options.  For example, Radio buttons, drop down, checkboxes, etc.
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Allows 10 rows per page, and starts on page 0. Columns and Rows may be selected but not deleted/edited
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


# Filter Radio button 
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#
#
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table

    # Water Rescue filter
        if filter_type == 'water':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # Mountain or Wilderness Rescue filter
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # Disaster Rescue or Individual Tracking filter
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset to no filter
        else:
            df = pd.DataFrame.from_records(animals.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     #Input('filter-type', 'value')
    ])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
#             figure = px.pie(dff, values='breed',  
#                             title='Available Dogs by Breed')
            figure = px.histogram(dff, x='breed')
        )    
    ]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available in the form of
# a dictionary. 
# derived_virtual_selected_row will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list. 
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_map(viewData):

    viewDF = pd.DataFrame.from_dict(viewData)
    
    dff = viewDF.loc[rows]
    
    return [
        
    # row_ids[0] gets last selected row from DataTable and parameters 13 and 14 are long and lat of rows  
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=15, children=[
            dl.TileLayer(id="base-layer-id"),
            
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                # parameter num 4 gives the breed of animal
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    # parameter 9 gives animal's name
                    html.P(dff.loc[0,'name']
                )
            ])
        ])
    ]


app.run_server(debug=True)

SyntaxError: invalid syntax (838535521.py, line 269)